# Analysing CIA Factbook Data Using SQL

In [2]:
%%capture
%load_ext sql
%sql sqlite:///Datasets/factbook.db

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///Datasets/factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT *
FROM facts
LIMIT 10

 * sqlite:///Datasets/factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.8
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


## Calculate Summary Statistics

In [6]:
%%sql
SELECT MIN(population), MAX(population),
MIN(population_growth), MAX(population_growth)
FROM facts

 * sqlite:///Datasets/factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Upon observinging the outputted summary statistics, two key values can be highlighted:
* There's a country with a population of 0
* There's a country with a population greater than 7.2 billion - likely representing the global population

In [8]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MIN(population)
                    FROM facts)

 * sqlite:///Datasets/factbook.db
Done.


name
Antarctica


In [9]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts)

 * sqlite:///Datasets/factbook.db
Done.


name
World


After assessing the values corresponding to the aforementioned highlighted summary statistics, both names are understandable. In order to find the country with the greatest population, this row will be removed.

In [12]:
%%sql
SELECT MIN(population), MAX(population),
MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != 'World'

 * sqlite:///Datasets/factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [15]:
%%sql
SELECT name
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts
                    WHERE name != 'World')

 * sqlite:///Datasets/factbook.db
Done.


name
China


In [16]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts

 * sqlite:///Datasets/factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Finding Densely Populated Countries

In [19]:
%%sql
SELECT name, population, area
FROM facts
WHERE (population > (SELECT AVG(population)
                     FROM facts)) AND 
       (area < (SELECT AVG(area)
                FROM facts))

 * sqlite:///Datasets/factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


Here, we can see countries that have a greater than average population alongside a smaller than average area. This therefore represents densely populated countries. 

## Finding the Country with the Highest Growth Rate

In [21]:
%%sql
SELECT name
FROM facts
WHERE population_growth = (SELECT MAX(population_growth)
                    FROM facts
                    WHERE name != 'World')

 * sqlite:///Datasets/factbook.db
Done.


name
South Sudan


## Countries with More Water Than Land

In [24]:
%%sql
SELECT name
FROM facts
WHERE area_water > area_land

 * sqlite:///Datasets/factbook.db
Done.


name
British Indian Ocean Territory
Virgin Islands


## Top 10 Highest Ratios of Water:Land

In [36]:
%%sql
SELECT name, ROUND(CAST(area_water AS Float) / CAST(area_land AS Float), 3) 'water_land_ratio', area_water, area_land, area
FROM facts
ORDER BY water_land_ratio DESC
LIMIT 10

 * sqlite:///Datasets/factbook.db
Done.


name,water_land_ratio,area_water,area_land,area
British Indian Ocean Territory,905.667,54340,60,54400
Virgin Islands,4.52,1564,346,1910
Puerto Rico,0.555,4921,8870,13791
"Bahamas, The",0.387,3870,10010,13880
Guinea-Bissau,0.285,8005,28120,36125
Malawi,0.259,24404,94080,118484
Netherlands,0.226,7650,33893,41543
Uganda,0.223,43938,197100,241038
Eritrea,0.164,16600,101000,117600
Liberia,0.156,15049,96320,111369
